# Defining transportation method: air, land, water.

## Conditions for aproximating the transportation method:

### Water:

- Importing country is an **island.**
- Importing country continent != partner country continent.

### Land:

- Importing country  **share border** with partner country .
- Same continent.
- Importing country  == partner country --> internal transport

### Air:

- Any country condition but **FOOD IS FRESH** (later)

In [1]:
import pandas as pd
import numpy as np

In [2]:
distance_df = pd.read_csv(r"..\data\FAOSTAT\clean_data\food_trading\country_matrix_v2.csv")

In [3]:
def get_transportation_method(row):
    if (row['any_island_or_missing'] == 1) or (row['same_continent'] == 0):
        return 'water'
    elif row['same_continent'] == 1 or (row['same_border'] == 1) or (row['distance_in_km'] == 0):
        return 'land'

In [4]:
distance_df['transportation_method'] = distance_df.apply(get_transportation_method, axis=1)

In [5]:
distance_df.head()

,country_from,continent_from,country_to,Continent_to,distance_in_km,same_continent,share_border,any_island_or_missing,transportation_method
0,afghanistan,asia,afghanistan,asia,0.000000,1,1.0,0,land
1,afghanistan,asia,albania,europe,4335.926901,0,0.0,0,water
2,afghanistan,asia,algeria,africa,5855.362822,0,0.0,0,water
3,afghanistan,asia,andorra,europe,5796.993207,0,NaN,1,water
4,afghanistan,asia,angola,africa,7596.963990,0,0.0,0,water


In [6]:
# Need to correct UK and ireland to be an island.
distance_df[distance_df['country_to'] == 'united kingdom']

,country_from,continent_from,country_to,Continent_to,distance_in_km,same_continent,share_border,any_island_or_missing,transportation_method
190,afghanistan,asia,united kingdom,europe,5707.952682,0,0.0,1,water
415,albania,europe,united kingdom,europe,1892.194281,1,0.0,1,water
640,algeria,africa,united kingdom,europe,1661.866330,0,0.0,1,water
865,andorra,europe,united kingdom,europe,1008.592159,1,NaN,1,water
1090,angola,africa,united kingdom,europe,6832.732677,0,0.0,1,water
...,...,...,...,...,...,...,...,...,...
49690,taiwan,asia,united kingdom,europe,9785.126246,0,0.0,1,water
49915,ussr,europe,united kingdom,europe,2500.558252,1,NaN,1,water
50140,venezuela,south america,united kingdom,europe,7498.554657,0,0.0,1,water
50365,wake island,oceania,united kingdom,europe,12037.346879,0,NaN,1,water


In [7]:
distance_df['transportation_method'].value_counts()

transportation_method
water    45123
land      5502
Name: count, dtype: int64

In [8]:
# Finally I will add country codes to distance matrix
country_codes = pd.read_csv(r"..\data\FAOSTAT\clean_data\fao_country_codes_corrected.csv", encoding="ISO-8859-1")

In [9]:
country_codes['Area'] = country_codes['Area'].str.lower()

In [10]:
distance_df = distance_df.merge(country_codes, how="left", left_on="country_from", right_on="Area")
distance_df = distance_df.rename(columns={'Area Code': 'country_from_code'})
distance_df.drop(columns='Area', inplace=True)

In [11]:
distance_df = distance_df.merge(country_codes, how="left", left_on="country_to", right_on="Area")
distance_df = distance_df.rename(columns={'Area Code': 'country_to_code'})
distance_df.drop(columns='Area', inplace=True)

In [16]:
distance_df['country_from_code'] = distance_df['country_from_code'].fillna(-1).astype(int)
distance_df['country_to_code'] = distance_df['country_to_code'].fillna(-1).astype(int)

In [17]:
distance_df.head()

,country_from,continent_from,country_to,Continent_to,distance_in_km,same_continent,share_border,any_island_or_missing,transportation_method,country_from_code,country_to_code
0,afghanistan,asia,afghanistan,asia,0.000000,1,1.0,0,land,2,2
1,afghanistan,asia,albania,europe,4335.926901,0,0.0,0,water,2,3
2,afghanistan,asia,algeria,africa,5855.362822,0,0.0,0,water,2,4
3,afghanistan,asia,andorra,europe,5796.993207,0,NaN,1,water,2,-1
4,afghanistan,asia,angola,africa,7596.963990,0,0.0,0,water,2,7


In [18]:
distance_df.to_csv(r"..\data\FAOSTAT\clean_data\food_trading\country_matrix_v3.csv", index=False)